<a href="https://colab.research.google.com/github/xvdp/siren/blob/x_dev/siren_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Siren Video stress test

paper mentions Supplementary Info pg. 16
"The Adam optimizer with a learning rate of1×10−4was used for all experi-ments. We set the batch size to fill the memory of the GPUs (roughly 160,000)"
"We train the videos for 100,000 iterations, requiring approximately 15 hours."


Batch size will depend on GPU availability
this branch 



In [1]:
!pip install git+https://github.com/xvdp/vidi.git
!rm -rf siren/
!git clone -b x_dev https://github.com/xvdp/siren.git

  Cloning https://github.com/xvdp/vidi.git to /tmp/pip-req-build-nofj6fan
  Running command git clone -q https://github.com/xvdp/vidi.git /tmp/pip-req-build-nofj6fan
  Created wheel for vidi: filename=vidi-0.1-cp37-none-any.whl size=25732 sha256=9bab229687b556d11fa120dec4ea8761555b1d3d575d737c385b9ec32cc8a2bc
  Stored in directory: /tmp/pip-ephem-wheel-cache-7u7ik90c/wheels/eb/af/bf/d3d553233ac4ba8401e2c03425734897d87fcb779db2b27f14
Successfully built vidi
Cloning into 'siren'...
remote: Enumerating objects: 290, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 290 (delta 55), reused 61 (delta 27), pack-reused 193
Receiving objects: 100% (290/290), 10.61 MiB | 16.84 MiB/s, done.
Resolving deltas: 100% (122/122), done.


In [2]:
import os
import os.path as osp
import yaml
import torch
os.chdir("siren")
import x_utils
import x_dataio
import x_modules
from x_training import train, _continue, load_last_checkpoint, _prevent_overwrite

## check available GPU and CPU

In [3]:
x_utils.GPUse(), x_utils.CPUse()

(GPU: ({'total': 15109, 'used': 0, 'available': 15109, 'percent': 0.0, 'units': 'MB'}),
 CPU: ({'total': 12993, 'used': 782, 'available': 12057, 'percent': 7.2, 'units': 'MB'}))

## download cat video
from https://drive.google.com/drive/u/0/folders/1_iq__37-hw7FJOEUK1tX7mdp8SKB368K

In [4]:
#https://drive.google.com/file/d/1ZCr6HTrNu8f6T-nyIbToYXHMOKU88f7P/view?usp=sharing
!gdown --id 1ZCr6HTrNu8f6T-nyIbToYXHMOKU88f7P 

Downloading...
From: https://drive.google.com/uc?id=1ZCr6HTrNu8f6T-nyIbToYXHMOKU88f7P
To: /content/siren/cat_video.mp4
4.49MB [00:00, 21.1MB/s]


In [5]:
os.rename("cat_video.mp4", "data/cat_video.mp4")

## connect gdrive to colab and redirect logging root

In [6]:
from google.colab import drive
drive_dir = '/content/gdrive'
drive.mount(drive_dir)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
logging_root = "/content/gdrive/MyDrive/siren"
os.makedirs(logging_root, exist_ok=True)
osp.isdir(logging_root)

True

In [8]:
# Easy_Dict is a dictionary accessible as object ( not pypi EasyDict )
config_file = "x_periment_scripts/cat_s-1_100k.yml"
with open(config_file, "r") as _fi:
    opt = x_utils.EasyDict(yaml.load(_fi,)) # Loader=yaml.FullLoader 

In [9]:
opt.logging_root = logging_root
config_file = "_colab".join(osp.splitext(config_file))
opt.to_yaml(config_file)
print(opt.logging_root, config_file, osp.isfile(config_file))

/content/gdrive/MyDrive/siren x_periment_scripts/cat_s-1_100k_colab.yml True


### training options differ from main branch: containing:
* model definition
* strategy  [-1] original, fully random single sample per epoch
 [1] complete set of non repeating samples per epoch [2] grided sampling, dense and sparse : original strategy works best
* sample_frac: None, is estimated from GPU available
* sample_size: estimated below


In [10]:
gpus = x_utils.GPUse()
# sample size is multiplied * 2 over the estimate, probably gradient operations over latents are optimized
# ergo single operations (sin, +, *) do not save full new latent gradient 
opt.sample_size = x_utils.estimate_samples(gpus.available, **opt["siren"])*2 
opt.epochs_til_checkpoint = 5000 # lower epochs till checkpoint, just in case, colab may boot us out

In [11]:
opt

{'batch_size': 1,
 'checkpoint_path': None,
 'data_path': './data/cat_video.mp4',
 'epochs_til_checkpoint': 5000,
 'experiment_name': 'cat_s-1_100k',
 'frame_range': None,
 'logging_root': '/content/gdrive/MyDrive/siren',
 'lr': 0.0001,
 'model_type': 'sine',
 'num_epochs': 100000,
 'num_steps': 100000,
 'sample_frac': None,
 'sample_size': 321482,
 'shuffle': 1,
 'siren': {'first_omega_0': 30,
  'hidden_features': 1024,
  'hidden_layers': 3,
  'hidden_omega_0': 30.0,
  'in_features': 3,
  'out_features': 3,
  'outermost_linear': True},
 'steps_til_summary': 1,
 'strategy': -1,
 'train_type': 'video',
 'verbose': 1}

## in this instance, sample_size is estimated at 321,480 float32
~2x the spec on the paper (160,000) 

**Does this mean that instead of 100k iterations similar loss will be reaches in 50k operations?**

Running the same experiments with the other strategies demonstrates that the a uniform random sampling (i.e. original strategy, -1), has  nearly monotonic convergence whereas running randomly repeating grids (strategy 2) or, non repeating complete random samples over the data ( strategy 1), overfits some data. 

These experimients will not be run here but to test wsuffices to set opt.strategy = 1 or 2



# VideoDataset is modificiation over main branch.
* Does not create m_grid then referenceit but finds grid pos from random index
* different strategies. see x_dataio.VideoDataset

In [12]:
dset = x_dataio.VideoDataset(opt.data_path, sample_size=opt.sample_size, frame_range=opt.frame_range, strategy=opt.strategy)

Siren: VideoDataset - INFO - Loaded data, sidelen: [300, 512, 512], channels 3
Siren: VideoDataset - INFO -          => reshaped to: (78643200, 3)
Siren: VideoDataset - INFO -  max sample_size, 321482, fraction, 0.0041
Siren: VideoDataset - INFO -  strategy: -1, single sample per epoch


In [13]:
dataloader = torch.utils.data.DataLoader(dset, shuffle=opt.shuffle, batch_size=1, pin_memory=True, num_workers=0)

## store training options to yaml

In [14]:
opt.sidelen = tuple(dset.data.shape[:-1])
opt.chanels = dset.data.shape[-1]
opt.sample_size = dset.sample_size
opt.dset_len = len(dset)
if "num_steps" not in opt:
    opt.num_steps = None
folder = osp.join(opt.logging_root, opt.experiment_name)
opt.to_yaml(osp.join(folder, "training_options.yml"))

## create model and run
Siren used in this instance is simplified - meta nodes disabled

In [15]:
model = x_modules.Siren(**opt["siren"])
model.cuda()

Siren(
  (net): Sequential(
    (0): SineLayer(
      (linear): Linear(in_features=3, out_features=1024, bias=True)
    )
    (1): SineLayer(
      (linear): Linear(in_features=1024, out_features=1024, bias=True)
    )
    (2): SineLayer(
      (linear): Linear(in_features=1024, out_features=1024, bias=True)
    )
    (3): SineLayer(
      (linear): Linear(in_features=1024, out_features=1024, bias=True)
    )
    (4): Linear(in_features=1024, out_features=3, bias=True)
  )
)

In [ ]:
checkpoint = train(model, dataloader, opt.num_epochs, opt.lr, opt.epochs_til_checkpoint, folder, dataset=dset, num_steps=opt.num_steps, terminator={"end":"\n"})

The model directory /content/gdrive/MyDrive/siren/cat_s-1_100k exists. Overwrite? (y/n)y
Epoch	Iter	IterAll	tGPU	GPU	CPU	Loss	Time	Total_Time
0	0	0	12622	13706	4157	0.27	0.82	0
1	0	1	13878	14962	4158	0.281	1.34	2
2	0	2	13878	14962	4158	0.178	1.52	4
3	0	3	13878	14962	4159	0.162	1.6	6
4	0	4	13878	14962	4159	0.14	1.66	8
5	0	5	13878	14962	4158	0.119	1.7	10
6	0	6	13878	14962	4159	0.105	1.72	12
7	0	7	13878	14962	4158	0.0961	1.75	13
8	0	8	13878	14962	4158	0.0915	1.76	15
9	0	9	13878	14962	4158	0.0878	1.78	17
10	0	10	13878	14962	4158	0.0823	1.79	19
11	0	11	13878	14962	4159	0.0774	1.8	21
12	0	12	13878	14962	4159	0.0746	1.81	23
13	0	13	13878	14962	4158	0.0697	1.81	25
14	0	14	13878	14962	4159	0.0679	1.82	27
15	0	15	13878	14962	4159	0.0652	1.83	29
16	0	16	13878	14962	4158	0.0626	1.84	31
17	0	17	13878	14962	4160	0.0602	1.84	33
18	0	18	13878	14962	4159	0.0584	1.85	35
19	0	19	13878	14962	4158	0.0567	1.85	37
20	0	20	13878	14962	4158	0.0545	1.85	38
21	0	21	13878	14962	4159	0.0534	1.86	40
22	0	22	13878	1